In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
new_version_path = Path(r'D:\vnpy-2.0.8')
# new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
# 加载模块
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

from datetime import datetime, timedelta
from typing import Tuple
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from boll_channel_strategy import BollChannelStrategy
from utility import (
    trade_zh_to_en,
    vt_bar_to_df,
    vt_trade_to_df,
    load_data,
    strip_digt,
    get_output_path,
    get_dominant_in_periods,
    clear_open_trade_after_deadline,
    single_backtest,
    get_pre_trading_date,
    strategy_dict,
    segment_backtest
)


### 进行分段合约的回测

- 这种方式只使用于合约在【变成非主力-到期】这个区间，一定会平仓离场自然切换的策略，基本上就是持仓周期较短的策略。
- 对于合约到期了还没有平仓信号的策略（如一直持仓的双均线策略），实盘的时候需要在到期前换到下个主力合约，在vnpy现有的回测引擎下，分段回测模拟这个换仓的逻辑会比较复杂，目前关心的策略持仓周期都在几天以内，从合约切换日日到上一个主力合约到期日之间的时间足够平仓切换了，所以需要换仓的问题先不涉及了。
- 结果对比发现，主连回测和分段回测在信号上出现的差异主要是集中在主力切换的那几天，中间段的数据基本是一致的。

In [ ]:
# 获取分段主力合约的开始和结束时间
commodity = 'rb'
interval = '1h'
strategy_name = 'turtle'
strategy_params = {}
capital = 100000
backtest_start = datetime(2018, 1, 1)
backtest_end = datetime(2019, 12, 1)

trade_df, pnl_df, res_df, res_dict = segment_backtest(commodity, interval, strategy_name, strategy_params, capital, backtest_start, backtest_end)